In [18]:
import pandas as pd
import numpy as np
import pycountry
import re

In [19]:
!pip install autocorrect

In [6]:
#data = pd.read_csv("/Users/davidcastrejon/Downloads/candyhierarchy2017.csv", encoding="latin")
data = pd.read_csv("candyhierarchy2017.csv", encoding="latin")
data.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


# Column 1

In [7]:
COLUMN_NAME = "Q1: GOING OUT?"
# convert to lowercase
data[COLUMN_NAME] = data[COLUMN_NAME].str.lower()
# Get count of yes, no, nan.
colOneValuesCount = data[COLUMN_NAME].astype(str).value_counts(dropna=False)
yesCountPercentage = colOneValuesCount.get("yes", 0) / (data[COLUMN_NAME].size - colOneValuesCount.get("nan", 0))
yesCountFill = round(yesCountPercentage * colOneValuesCount.get("nan", 0))
# shuffle the data frame
data = data.sample(frac=1).reset_index(drop=True)
   # fill our nan values with yes according to the countFill
try:
    data[COLUMN_NAME].fillna("yes", limit=yesCountFill, inplace=True)
    data[COLUMN_NAME].fillna("no", inplace=True)
    data[COLUMN_NAME].unique()
except:
    pass
colOneValuesCount

# we can use regex validation for an extra layer of security (good for data integrity for large data sets)

# define reg pattern to use
regexPattern = r"^(yes|no)$"

# parse through data to check if answers match regexPattern
def process_going_out(going_out):
    if re.match(regexPattern, going_out):
        return going_out
    else:
        return 'no'

data[COLUMN_NAME] = data[COLUMN_NAME].apply(process_going_out)
print(data[COLUMN_NAME].unique())
data[COLUMN_NAME] = data[COLUMN_NAME].map({'yes': 1, 'no': 0})


['yes' 'no']


## Column 2

In [8]:
COLUMN_NAME = "Q2: GENDER"  # Replace with the actual column name in your DataFrame

# Convert to lowercase and fill NA values
data[COLUMN_NAME] = data[COLUMN_NAME].str.lower()
data[COLUMN_NAME].fillna("other", inplace=True)

# Define regex pattern
regexPattern = r"^(female|male|other|i'd rather not say)$"

# Function to process gender
def process_gender(gender):
    gender_str = str(gender)
    match = re.match(regexPattern, gender_str)
    if match:
        return match.group()
    else:
        return 'other'

# Apply the function to the gender column
data[COLUMN_NAME] = data[COLUMN_NAME].apply(process_gender)

# One-hot encode
gender_dummies = pd.get_dummies(data[COLUMN_NAME], prefix='gender')
data = pd.concat([data, gender_dummies], axis=1)

# Drop the original column
data.drop([COLUMN_NAME], axis=1, inplace=True)
print(data)



      Internal ID  Q1: GOING OUT? Q3: AGE    Q4: COUNTRY  \
0        90276841               1      49            usa   
1        90286810               0      37  United States   
2        90304431               0      19  united states   
3        90277094               0      36         Canada   
4        90287801               1      34            USA   
...           ...             ...     ...            ...   
2455     90284966               0      75          U.S.    
2456     90275199               1      63            usa   
2457     90280664               0      63             us   
2458     90275444               0      55            Usa   
2459     90287047               0      48  United States   

     Q5: STATE, PROVINCE, COUNTY, ETC Q6 | 100 Grand Bar  \
0                                  mn                JOY   
1                             Indiana                NaN   
2                               texas                JOY   
3                             Alberta  

# Column 3

In [9]:
COLUMN_NAME = "Q3: AGE"
# Unique
#for the text data in the data frame that is age we can use reguaar expression to find the numbers and then convert them to numeric values
#
#
"""
[nan, '44', '49', '40', '23', '53', '33', '43', '56', '64', '37',
       '59', '48', '54', '36', '45', '25', '34', '35', '38', '58', '50',
       '47', '16', '52', '63', '65', '41', '27', '31', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '26', '51',
       '70', '24', '18', '19', 'Old enough', '57', '60', '66', '12',
       'Many', '55', '72', '?', '21', '11', 'no', '9', '68', '20', '6',
       '10', '71', '90', '13', '45-55', '312', '99', '7', 'hahahahaha',
       '88', '39.4', '74', '102', 'older than dirt', '17', '15', '8',
       '75', '5u', 'Enough', 'See question 2', '24-50', '14', 'Over 50',
       '100', '76', '1000', 'sixty-nine', '46 Halloweens.', 'ancient',
       '77', 'OLD', 'old', '73', '70 1/2', '1', 'MY NAME JEFF', '4',
       '59 on the day after Halloween', 'old enough', 'your mom',
       'I can remember when Java was a cool new language', '60+']
"""

# Replace series as a numeric type rounded to the nearest whole number.
data[COLUMN_NAME] = pd.to_numeric(data[COLUMN_NAME], errors='coerce').round() # convert to numeric values & round

# remove columns which have values not in range 5-100
data[COLUMN_NAME] = data[COLUMN_NAME][(data[COLUMN_NAME] >= 5) & (data[COLUMN_NAME] <= 100)]  # filter to only ages of 5-100
# subsetcolumn label or sequence of labels, optional
data.dropna(subset=[COLUMN_NAME], inplace=True) # drop all rows with nan values
data[COLUMN_NAME]


# double check if there are any non-numeric values, this should be empty:
non_numeric_values = data[~data[COLUMN_NAME].apply(lambda x: np.isreal(x))]
message = "number of non-numeric values in column " + COLUMN_NAME + " (should be empty):"
print(message, non_numeric_values)

# fill nil values with the median of the age column (or mean if you want, i commented that out)
data[COLUMN_NAME].fillna(data[COLUMN_NAME].median(), inplace=True)
# data[COLUMN_NAME].fillna(data[COLUMN_NAME].mean(), inplace=True)

# double check to see if any nulls are left
message = "number of nulls in column " + COLUMN_NAME + " (should be 0):"
print (message, data[COLUMN_NAME].isna().sum())


print(data[COLUMN_NAME]);

number of non-numeric values in column Q3: AGE (should be empty): Empty DataFrame
Columns: [Internal ID, Q1: GOING OUT?, Q3: AGE, Q4: COUNTRY, Q5: STATE, PROVINCE, COUNTY, ETC, Q6 | 100 Grand Bar, Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes), Q6 | Any full-sized candy bar, Q6 | Black Jacks, Q6 | Bonkers (the candy), Q6 | Bonkers (the board game), Q6 | Bottle Caps, Q6 | Box'o'Raisins, Q6 | Broken glow stick, Q6 | Butterfinger, Q6 | Cadbury Creme Eggs, Q6 | Candy Corn, Q6 | Candy that is clearly just the stuff given out for free at restaurants, Q6 | Caramellos, Q6 | Cash, or other forms of legal tender, Q6 | Chardonnay, Q6 | Chick-o-Sticks (we donÕt know what that is), Q6 | Chiclets, Q6 | Coffee Crisp, Q6 | Creepy Religious comics/Chick Tracts, Q6 | Dental paraphenalia, Q6 | Dots, Q6 | Dove Bars, Q6 | Fuzzy Peaches, Q6 | Generic Brand Acetaminophen, Q6 | Glow sticks, Q6 | Goo Goo Clusters, Q6 | Good N' Plenty, Q6 | Gum from baseball cards, Q6 | Gu

# Column 4


In [17]:
import pycountry
from autocorrect import Speller
# use the pycountry and autocorrect libraries
COLUMN_NAME = "Q4: COUNTRY"

# initialize Speller obj from library
spell = Speller()



# function to autocorrect country names
def autocorrectCountries(value):
    value=value.lower()
    orig=value
    corrected_country = spell(value) # autocorrect name
    try:
        country = pycountry.countries.lookup(corrected_country) # look up country from pycountry
        if country:
            return country.name.lower() # if found, return the (lowercased) standard spelling
    except Exception as e:
        return orig              # else return nan (not sure if there's something better?)

def standardize_country_name(name):
    name = name.lower().strip()

    # Remapping for United States
    if any(keyword in name for keyword in ['us', 'u.s.', 'america', 'states', 'us of a', 'unhinged states', 'usa ', 'united state', 'unite states', "'merica", 'usausausa', 'u s ', 'u.s. ', 'u s a', "usa? hard to tell anymore..", "i don't know anymore", 'north carolina', 'new york', 'california', 'pittsburgh', 'new jersey', 'n. america', 'usa usa usa!!!!', 'usa! usa! usa!', 'murica', 'murrika']):
        return 'united states'

    # Remapping for United Kingdom
    if any(keyword in name for keyword in ['uk', 'u.k.', 'england', 'united kingdom', 'scotland', 'uk ', 'u.k. ', 'endland']):
        return 'united kingdom'

    # Remapping for Canada
    if any(keyword in name for keyword in ['canada', 'soviet canuckistan', 'i pretend to be from canada, but i am really from the united states.', 'canada ', 'canae', 'canada`']):
        return 'canada'

    # Remapping for United Arab Emirates
    if 'united arab emirates' in name:
        return 'united arab emirates'

    # Remapping for China
    if any(keyword in name for keyword in ['china', 'taiwan, province of china', 'hong kong']):
        return 'china'

    # ... additional remapping rules
    if 'mexico' in name:
        return 'mexico'

    if 'iceland' in name:
        return 'iceland'

    if 'germany' in name:
        return 'germany'

    if any(keyword in name for keyword in ['netherlands', 'the netherlands']):
        return 'netherlands'

    if 'denmark' in name:
        return 'denmark'

    if any(keyword in name for keyword in ['ireland', 'ireland ']):
        return 'ireland'

    if 'indonesia' in name:
        return 'indonesia'

    if any(keyword in name for keyword in ['japan', 'korea', 'korea, republic of']):
        return 'japan/korea'

    if 'europe' in name:
        return 'europe'

    if 'switzerland' in name:
        return 'switzerland'

    if 'spain' in name:
        return 'spain'

    if any(keyword in name for keyword in ['france', 'france ']):
        return 'france'

    if 'sweden' in name:
        return 'sweden'

    if 'trumpistan' in name:
        return 'fictional/cultural reference'

    if 'finland' in name:
        return 'finland'

    if 'greece' in name:
        return 'greece'

    if 'south africa' in name:
        return 'south africa'

    if 'costa rica' in name:
        return 'costa rica'

    if 'singapore' in name:
        return 'singapore'

    if 'earth' in name:
        return 'earth'

    if 'narnia' in name or 'cascadia' in name or 'ud' in name:
        return 'fictional/cultural reference'

    if 'nan' in name:
        return 'unknown'

    # If name doesn't match any known patterns, keep it as is
    return 'other'

# Assuming data is your DataFrame
COLUMN_NAME = "Q4: COUNTRY"
data[COLUMN_NAME] = data[COLUMN_NAME].astype(str).apply(standardize_country_name)

# Now print unique values to verify
print(data[COLUMN_NAME].value_counts())

    
# apply to each country in column and retrieve all the unique names
# data[COLUMN_NAME] = data[COLUMN_NAME].astype(str).apply(autocorrectCountries)
# data[COLUMN_NAME].unique()


united states     2011
canada             222
united kingdom      39
other               18
germany             10
netherlands          7
japan/korea          7
china                4
ireland              4
mexico               4
switzerland          3
france               3
sweden               2
denmark              2
south africa         1
costa rica           1
finland              1
earth                1
greece               1
europe               1
iceland              1
indonesia            1
singapore            1
spain                1
Name: Q4: COUNTRY, dtype: int64


# Column 5

- ### Oppenheimer this column 💣

In [ ]:
data.drop(columns=['Q5: STATE, PROVINCE, COUNTY, ETC'],inplace=True)

In [ ]:
data.head()

# Columns 5 - 97

In [ ]:
start_index = 5
end_index = 106

# Use list slicing to extract column names into a list
sequential_column_names = data.columns[start_index:end_index + 1].tolist()

for col in sequential_column_names:
    # Use the 'replace' method to map values to '001', '010', '100'; Applying Hot Encoding
    data[col] = data[col].replace({'JOY': '00001', 'MEH': '00010', 'MEH': '00100', 'DESPAIR': '01000', np.nan: '10000'})
    data[col] = data[col].replace({'00001': 0, '00010': 1, '00100': 2, '01000': 3, '10000': 4})

# Show the first few rows of the data frame
# data.head()
data.iloc[:, start_index:end_index].head()

In [ ]:
data['Q3: AGE'].median()

# Column 111?

In [ ]:
COLUMN_NAME = "Q10: DRESS"

def hot_encoding_values(df, col):
    nominal_values = df[col].unique().tolist()
    
    encoding_vals = {elem: idx for idx, elem in enumerate(nominal_values)}
    
    # can return the hashmap to see exactly what pertains to what as well. 
    
    df[col].replace(encoding_vals, inplace=True)
    print(df[col])
    

hot_encoding_values(data, COLUMN_NAME)

# Column 112
- ### Oppenheimer this column 💣

In [ ]:
# drop the empty cell column. 
data.drop(data.columns[112], axis=1, inplace=True)
data.iloc[:, 112:113].head() # assuring the column was dropped

# Column 112 

- ### This column is now 112, in the OG set it is 113. However we dropped an empty column with no useful information above

In [ ]:
# These are nominal values as well so we can in theory apply hot encoding to these values pertaining to the date
COLUMN_NAME = "Q11: DAY"
COLUMN_NAME = data.columns[112]
print(COLUMN_NAME)

    
    
# encode the vals
hot_encoding_values(data, COLUMN_NAME)